In [10]:
import pandas as pd
import sqlite3

<h2>Создайте подключение к базе данных с помощью библиотеки sqlite3</h2>

In [11]:
conn = sqlite3.connect('../data/checking-logs.sqlite')
conn

In [12]:
pd.io.sql.read_sql('select * from checker limit 5', conn)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1


In [13]:
pd.io.sql.read_sql('select * from pageviews limit 5', conn)

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665


<h2>создайте новую таблицу в базе данных, объединив таблицы «Просмотр страниц» и «Проверка» с помощью всего одного запроса</h2>

- таблица должна иметь следующие столбцы: uid, labname, first_commit_ts, first_view_ts.
- first_commit_ts — это просто новое имя временной метки столбца из таблицы проверки, оно показывает первый коммит из конкретной лаборатории и от конкретного пользователя.
- first_view_ts — первое посещение пользователем таблицы просмотров страниц, временная метка, когда пользователь посетил ленту новостей.
- status = 'готово' все равно должен быть фильтром
- numTrials = 1 все равно должно быть фильтром
- имена лабораторий по-прежнему должны быть из списка: «laba04», «laba04s», «laba05», «laba06», «laba06s», «project1».
- таблица должна содержать только пользователей (uid с user_*), а не администраторов
- first_commit_ts и first_view_ts следует анализировать как datetime64[ns]

In [14]:
query = """
SELECT checker.uid,
       checker.labname,
       checker.timestamp as first_commit_ts,
       pageviews.datetime as first_view_ts
FROM 
    checker
LEFT JOIN 
    pageviews 
ON 
    checker.uid=pageviews.uid
WHERE 
    status='ready'
AND numTrials=1
AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
AND checker.uid LIKE 'user_%'
AND (pageviews.datetime = (SELECT MIN(pageviews.datetime)
                           FROM pageviews
                           WHERE uid=checker.uid)
                    OR pageviews.datetime IS NULL)
"""
data = pd.io.sql.read_sql(query, conn, parse_dates = ['first_commit_ts', 'first_view_ts'])
data

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


<h2>Используя методы Pandas, создайте два фрейма данных: тестовый и контрольный</h2>

- у теста должны быть пользователи, у которых есть значения в first_view_ts
- у элемента управления должны быть пользователи, у которых в first_view_ts отсутствуют значения
- замените отсутствующие значения в контрольной группе средним значением first_view_ts тестовых пользователей, мы будем использовать это значение для дальнейшего анализа
- сохраните обе таблицы в базе данных, вы будете использовать их в следующих упражнениях

In [15]:
test = data[data['first_view_ts'].notnull()]
control = data[data['first_view_ts'].isnull()]
control = control.fillna(test['first_view_ts'].mean())

In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [17]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [21]:
#сохранение
control.to_sql('control', conn, if_exists='replace')  # Замена таблицы, если она существует
control.to_sql('control', conn, if_exists='replace')

81

<h2>Закрытие соединения</h2>

In [19]:
conn.close()